<a href="https://colab.research.google.com/github/tarumi283/tarumi/blob/main/Bonferroni_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

pip install pandas statsmodels

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multitest import multipletests

# CSVファイルのパス
csv_file_path = 'path_to_your_file.csv'

# CSVファイルの読み込み
data = pd.read_csv(csv_file_path)

# データの確認
print("データの先頭部分:\n", data.head())

# グループ列と値列の名前を適切に設定してください
group_column = 'group'  # グループを示す列名
value_column = 'value'  # 値を示す列名

# ANOVAの実行
model = ols(f'{value_column} ~ {group_column}', data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("\nANOVAの結果:\n", anova_table)

# グループごとに値のリストを作成
groups = data[group_column].unique()
group_data = [data[data[group_column] == group][value_column] for group in groups]

# t検定を用いてすべてのペアのp値を計算
from itertools import combinations
from scipy.stats import ttest_ind

p_values = []
comparisons = list(combinations(groups, 2))
for (group1, group2) in comparisons:
    t_stat, p_value = ttest_ind(data[data[group_column] == group1][value_column],
                                data[data[group_column] == group2][value_column])
    p_values.append(p_value)

# ボンフェローニ法を適用してp値を補正
reject, pvals_corrected, alphacSidak, alphacBonf = multipletests(p_values, alpha=0.05, method='bonferroni')

# 結果の表示
print("\nボンフェローニ補正後のp値:")
for i, (comp, p_val_corr) in enumerate(zip(comparisons, pvals_corrected)):
    print(f"{comp[0]} vs {comp[1]}: {p_val_corr} (補正前のp値: {p_values[i]})")

print("\n補正されたp値で有意な差があるペア:")
for i, (comp, rej) in enumerate(zip(comparisons, reject)):
    if rej:
        print(f"{comp[0]} vs {comp[1]}")